In [1]:
# import sys

import numpy as np
import pandas as pd
import random
from warnings import warn
from collections import OrderedDict

from aif360.datasets import GermanDataset, StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
# from common_utils import compute_metrics
# from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        # import load_preproc_data_german
from data_preproc_functions import load_preproc_data_german
from sklearn.preprocessing import MaxAbsScaler
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.inprocessing import MetaFairClassifier
from aif360.algorithms.postprocessing import RejectOptionClassification


from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display

2024-05-01 15:29:09.385729: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/jessicachen/opt/anaconda3/envs/226/lib/python3.8/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


In [21]:
def compute_metrics(dataset_true, dataset_pred, 
                    unprivileged_groups, privileged_groups,
                    disp = True):
    """ Compute the key metrics """
    classified_metric_pred = ClassificationMetric(dataset_true,
                                                 dataset_pred, 
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
    metrics = OrderedDict()
    metrics["Balanced accuracy"] = 0.5*(classified_metric_pred.true_positive_rate()+
                                             classified_metric_pred.true_negative_rate())
    metrics["Statistical parity difference"] = classified_metric_pred.statistical_parity_difference()
    metrics["Disparate impact"] = classified_metric_pred.disparate_impact()
    metrics["Average odds difference"] = classified_metric_pred.average_odds_difference()
    metrics["Equal opportunity difference"] = classified_metric_pred.equal_opportunity_difference()
    metrics["Theil index"] = classified_metric_pred.theil_index()
    metrics["Consistency (individual fairness)"] = classified_metric_pred.consistency()
    metrics["Generalized entropy index (unified individual and group fairness)"] = classified_metric_pred.generalized_entropy_index()
    
    # if disp:
    #     for k in metrics:
    #         print("%s = %.4f" % (k, metrics[k]))
    
    return metrics

In [3]:
def load_dropped_data_german(protected_attributes=None):
    dataset = load_preproc_data_german()
    df = dataset.convert_to_dataframe()[0]
    df = df[['age','credit_history=Other','savings=<500','credit']]

    # Feature partitions
    # XD_features = ['credit_history', 'savings', 'employment', 'sex', 'age']
    D_features = ['sex', 'age'] if protected_attributes is None else protected_attributes
    Y_features = ['credit']
    # X_features = list(set(XD_features)-set(D_features))
    # print(X_features)
    categorical_features = ['credit_history', 'savings', 'employment']

    # privileged classes
    all_privileged_classes = {"sex": [1.0],
                              "age": [1.0]}

    # protected attribute maps
    all_protected_attribute_maps = {"sex": {1.0: 'Male', 0.0: 'Female'},
                                    "age": {1.0: 'Old', 0.0: 'Young'}}
    new_german_data = StandardDataset(
            df=df,
            label_name=Y_features[0],
            favorable_classes=[1],
            protected_attribute_names=D_features,
            privileged_classes=[all_privileged_classes[x] for x in D_features],
            instance_weights_name=None,
            features_to_keep=['credit_history=Other','savings=<500']+Y_features+D_features,
            metadata={ 'label_maps': [{1.0: 'Good Credit', 2.0: 'Bad Credit'}],
                    'protected_attribute_maps': [all_protected_attribute_maps[x]
                                    for x in D_features]})
    
    return new_german_data

In [4]:
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

In [5]:
# DROPPED DATA
dataset_orig = load_dropped_data_german(['age'])

# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig.favorable_label, dataset_orig.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig.privileged_protected_attributes, 
      dataset_orig.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig.feature_names)
display(Markdown("#### Dataset label"))
print(dataset_orig.label_names)

['credit_history', 'savings', 'employment']
0      A34
1      A32
2      A34
3      A32
4      A33
      ... 
995    A32
996    A32
997    A32
998    A32
999    A34
Name: credit_history, Length: 1000, dtype: object
0          Other
1      None/Paid
2          Other
3      None/Paid
4          Delay
         ...    
995    None/Paid
996    None/Paid
997    None/Paid
998    None/Paid
999        Other
Name: credit_history, Length: 1000, dtype: object


#### Training Dataset shape

(1000, 3)


#### Favorable and unfavorable labels

1.0 2.0


...

[array([1.])] [array([0.])]


#### Dataset feature names

['age', 'credit_history=Other', 'savings=<500']


#### Dataset label

['credit']


In [6]:
# split data into epochs, each with a different group of agents
NUM_EPOCHS = 2
dataset_orig_epochs = dataset_orig.split(NUM_EPOCHS, shuffle=True)
print("Size of epoch: ", dataset_orig_epochs[0].features.shape)

Size of epoch:  (500, 3)


In [7]:
# NO FAIRNESS
# takes two epochs of data
# trains on epoch_1, then uses model to classify epoch 2

def no_fairness_train_and_classify(data_epoch_1, data_epoch_2):
    # print out some labels, names, etc.
    # display(Markdown("#### No Fairness"))
    # print("Epoch 1: ", data_epoch_1.features.shape)
    # print("Epoch 2: ",data_epoch_2.features.shape)

    # train classifier on epoch 1
    scale_orig = StandardScaler()
    X_train = scale_orig.fit_transform(data_epoch_1.features)
    y_train = data_epoch_1.labels.ravel()
    lmod = LogisticRegression(solver='liblinear')  # Solver specified to avoid future warnings
    lmod.fit(X_train, y_train)

    # classify epoch 2 agents
    X_epoch2 = scale_orig.fit_transform(data_epoch_2.features)
    y_epoch2_pred = lmod.predict(X_epoch2)
    data_epoch_2_pred = data_epoch_2.copy(deepcopy=True)
    data_epoch_2_pred.labels = y_epoch2_pred

    # print("Classifications: ", data_epoch_2_pred.labels)

    # Evaluate fairness metrics on classification of epoch 2
    metric_train = BinaryLabelDatasetMetric(data_epoch_2_pred, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
    
    # print("Training set: Difference in mean outcomes = {:.3f}".format(metric_train.mean_difference()))

    metric_test_aft = compute_metrics(data_epoch_2, data_epoch_2_pred, 
                unprivileged_groups, privileged_groups)
    
    # The estimated coefficients will all be around 1:
    # print(lmod.coef_)
    # print(data_epoch_1.feature_names)

    return lmod, metric_test_aft

In [8]:
def RW_train_and_classify(data_epoch_1, data_epoch_2):
    # reweigh epoch 1 data
    # display(Markdown("#### Reweighing Fairness"))

    RW1 = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
    RW1.fit(data_epoch_1)
    dataset_transf_train1 = RW1.transform(data_epoch_1)

    # train classifier on epoch 1
    scale_orig = StandardScaler()
    X_train = scale_orig.fit_transform(dataset_transf_train1.features)
    y_train = dataset_transf_train1.labels.ravel()
    lmod = LogisticRegression(solver='liblinear')  # Solver specified to avoid future warnings
    lmod.fit(X_train, y_train)

    # reweigh epoch 2 data
    RW2 = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
    RW2.fit(data_epoch_2)
    dataset_transf_train2 = RW2.transform(data_epoch_2)

    # classify epoch 2 agents
    X_epoch2 = scale_orig.fit_transform(dataset_transf_train2.features)
    y_epoch2_pred = lmod.predict(X_epoch2)
    data_epoch_2_pred = dataset_transf_train2.copy(deepcopy=True)
    data_epoch_2_pred.labels = y_epoch2_pred

    # print("Classifications: ", data_epoch_2_pred.labels)

    # Evaluate fairness metrics on classification of epoch 2
    metric_train = BinaryLabelDatasetMetric(data_epoch_2_pred, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
    
    # print("Training set: Difference in mean outcomes = {:.3f}".format(metric_train.mean_difference()))

    metric_test_aft = compute_metrics(dataset_transf_train2, data_epoch_2_pred, 
                unprivileged_groups, privileged_groups)
    
    # The estimated coefficients will all be around 1:
    # print(lmod.coef_)
    # print(data_epoch_1.feature_names)

    return lmod, metric_test_aft

In [9]:
RW_train_and_classify(dataset_orig_epochs[0],dataset_orig_epochs[1])

Balanced accuracy = 0.5174
Statistical parity difference = -0.5259
Disparate impact = 0.4741
Average odds difference = -0.5627
Equal opportunity difference = -0.4783
Theil index = 0.1228
Consistency (individual fairness) = 0.6844
Generalized entropy index (unified individual and group fairness) = 0.0999


(LogisticRegression(solver='liblinear'),
 OrderedDict([('Balanced accuracy', 0.5173861892583119),
              ('Statistical parity difference', -0.5258618925831203),
              ('Disparate impact', 0.4741381074168797),
              ('Average odds difference', -0.5626598465473145),
              ('Equal opportunity difference', -0.4782608695652174),
              ('Theil index', 0.12280542313403064),
              ('Consistency (individual fairness)', array([0.6844])),
              ('Generalized entropy index (unified individual and group fairness)',
               0.09989050542231129)]))

In [10]:
def ROC_train_and_classify(data_epoch_1, data_epoch_2):
    # print out some labels, names, etc.
    # display(Markdown("#### ROC Fairness"))
    # print("Epoch 1: ", data_epoch_1.features.shape)
    # print("Epoch 2: ",data_epoch_2.features.shape)

    # Metric used (should be one of allowed_metrics)
    metric_name = "Statistical parity difference"

    # Upper and lower bound on the fairness metric used
    metric_ub = 0.05
    metric_lb = -0.05

    scale_orig = StandardScaler()

    # need to first train a model to get predicted scores
    X_train = scale_orig.fit_transform(data_epoch_1.features)
    y_train = data_epoch_1.labels.ravel()
    lmod = LogisticRegression(solver='liblinear')  # Solver specified to avoid future warnings
    lmod.fit(X_train, y_train)

    # indices of favorable label
    pos_ind = np.where(lmod.classes_ == data_epoch_1.favorable_label)[0][0]

    # data_epoch_1_pred contains PREDICTED SCORES
    # use same epoch 1 data instead of separate validation
    data_epoch_1_pred = data_epoch_1.copy(deepcopy=True)
    X_train = scale_orig.transform(data_epoch_1_pred.features)
    data_epoch_1_pred.scores = lmod.predict_proba(X_train)[:,pos_ind].reshape(-1,1)

    ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                  num_class_thresh=100, num_ROC_margin=50,
                                  metric_name=metric_name,
                                  metric_ub=metric_ub, metric_lb=metric_lb)
    ROC = ROC.fit(data_epoch_1, data_epoch_1_pred)

    # print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
    # print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

    # Metrics for the transformed test set
    data_epoch_2_pred = ROC.predict(data_epoch_2)

    # Evaluate fairness metrics on classification of epoch 2
    metric_train = BinaryLabelDatasetMetric(data_epoch_2_pred, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
    
    # print("Training set: Difference in mean outcomes = {:.3f}".format(metric_train.mean_difference()))

    metric_test_aft = compute_metrics(data_epoch_2, data_epoch_2_pred, 
                unprivileged_groups, privileged_groups)
    
    return lmod, metric_test_aft

In [11]:
def RW_ROC_train_and_classify(data_epoch_1, data_epoch_2):
    # print out some labels, names, etc.
    # display(Markdown("#### RW ROC Fairness"))
    # print("Epoch 1: ", data_epoch_1.features.shape)
    # print("Epoch 2: ",data_epoch_2.features.shape)

    # Metric used (should be one of allowed_metrics)
    metric_name = "Statistical parity difference"

    # Upper and lower bound on the fairness metric used
    metric_ub = 0.05
    metric_lb = -0.05

    # reweigh epoch 1 data
    RW1 = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
    RW1.fit(data_epoch_1)
    dataset_transf_train1 = RW1.transform(data_epoch_1)

    scale_orig = StandardScaler()

    # need to first train a model to get predicted scores
    X_train = scale_orig.fit_transform(dataset_transf_train1.features)
    y_train = dataset_transf_train1.labels.ravel()
    lmod = LogisticRegression(solver='liblinear')  # Solver specified to avoid future warnings
    lmod.fit(X_train, y_train)

    # indices of favorable label
    pos_ind = np.where(lmod.classes_ == dataset_transf_train1.favorable_label)[0][0]

    # data_epoch_1_pred contains PREDICTED SCORES
    # use same epoch 1 data instead of separate validation
    data_epoch_1_pred = dataset_transf_train1.copy(deepcopy=True)
    X_train = scale_orig.transform(data_epoch_1_pred.features)
    data_epoch_1_pred.scores = lmod.predict_proba(X_train)[:,pos_ind].reshape(-1,1)

    ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                  num_class_thresh=100, num_ROC_margin=50,
                                  metric_name=metric_name,
                                  metric_ub=metric_ub, metric_lb=metric_lb)
    ROC = ROC.fit(dataset_transf_train1, data_epoch_1_pred)

    # print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
    # print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

    # reweigh epoch 1 data
    RW2 = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
    RW2.fit(data_epoch_2)
    dataset_transf_train2 = RW1.transform(data_epoch_2)

    # Metrics for the transformed test set
    data_epoch_2_pred = ROC.predict(dataset_transf_train2)

    # Evaluate fairness metrics on classification of epoch 2
    metric_train = BinaryLabelDatasetMetric(data_epoch_2_pred, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
    
    # print("Training set: Difference in mean outcomes = {:.3f}".format(metric_train.mean_difference()))

    metric_test_aft = compute_metrics(dataset_transf_train2, data_epoch_2_pred, 
                unprivileged_groups, privileged_groups)
    
    return lmod, metric_test_aft

In [12]:
no_fairness_train_and_classify(dataset_orig_epochs[0],dataset_orig_epochs[1])

Balanced accuracy = 0.5321
Statistical parity difference = -0.5340
Disparate impact = 0.4660
Average odds difference = -0.5627
Equal opportunity difference = -0.4783
Theil index = 0.1228
Consistency (individual fairness) = 0.6844
Generalized entropy index (unified individual and group fairness) = 0.0999


(LogisticRegression(solver='liblinear'),
 OrderedDict([('Balanced accuracy', 0.5320531816116478),
              ('Statistical parity difference', -0.5339805825242718),
              ('Disparate impact', 0.46601941747572817),
              ('Average odds difference', -0.5626598465473145),
              ('Equal opportunity difference', -0.4782608695652174),
              ('Theil index', 0.12280542313403064),
              ('Consistency (individual fairness)', array([0.6844])),
              ('Generalized entropy index (unified individual and group fairness)',
               0.09989050542231129)]))

In [13]:
ROC_train_and_classify(dataset_orig_epochs[0],dataset_orig_epochs[1])

Balanced accuracy = 1.0000
Statistical parity difference = -0.0606
Disparate impact = 0.9171
Average odds difference = 0.0000
Equal opportunity difference = 0.0000
Theil index = 0.0000
Consistency (individual fairness) = 0.6844
Generalized entropy index (unified individual and group fairness) = 0.0000


(LogisticRegression(solver='liblinear'),
 OrderedDict([('Balanced accuracy', 1.0),
              ('Statistical parity difference', -0.06057567679929565),
              ('Disparate impact', 0.9170739872782057),
              ('Average odds difference', 0.0),
              ('Equal opportunity difference', 0.0),
              ('Theil index', 0.0),
              ('Consistency (individual fairness)', array([0.6844])),
              ('Generalized entropy index (unified individual and group fairness)',
               0.0)]))

In [14]:
RW_train_and_classify(dataset_orig_epochs[0],dataset_orig_epochs[1])

Balanced accuracy = 0.5174
Statistical parity difference = -0.5259
Disparate impact = 0.4741
Average odds difference = -0.5627
Equal opportunity difference = -0.4783
Theil index = 0.1228
Consistency (individual fairness) = 0.6844
Generalized entropy index (unified individual and group fairness) = 0.0999


(LogisticRegression(solver='liblinear'),
 OrderedDict([('Balanced accuracy', 0.5173861892583119),
              ('Statistical parity difference', -0.5258618925831203),
              ('Disparate impact', 0.4741381074168797),
              ('Average odds difference', -0.5626598465473145),
              ('Equal opportunity difference', -0.4782608695652174),
              ('Theil index', 0.12280542313403064),
              ('Consistency (individual fairness)', array([0.6844])),
              ('Generalized entropy index (unified individual and group fairness)',
               0.09989050542231129)]))

In [15]:
RW_ROC_train_and_classify(dataset_orig_epochs[0],dataset_orig_epochs[1])

Balanced accuracy = 1.0000
Statistical parity difference = 0.1436
Disparate impact = 1.2091
Average odds difference = 0.0000
Equal opportunity difference = 0.0000
Theil index = 0.0000
Consistency (individual fairness) = 0.6844
Generalized entropy index (unified individual and group fairness) = 0.0000


(LogisticRegression(solver='liblinear'),
 OrderedDict([('Balanced accuracy', 1.0),
              ('Statistical parity difference', 0.14356030392477792),
              ('Disparate impact', 1.2091303827020081),
              ('Average odds difference', 0.0),
              ('Equal opportunity difference', 0.0),
              ('Theil index', 0.0),
              ('Consistency (individual fairness)', array([0.6844])),
              ('Generalized entropy index (unified individual and group fairness)',
               0.0)]))

In [16]:
def append_dataset_history(dataset_history, new_epoch, protected_attributes=None):
    history_df = dataset_history.convert_to_dataframe()[0]
    epoch_df = new_epoch.convert_to_dataframe()[0]

    history_df = pd.concat([history_df,epoch_df])

    # CONVERT DATASET
    D_features = ['sex', 'age'] if protected_attributes is None else protected_attributes
    Y_features = ['credit']
    # X_features = list(set(XD_features)-set(D_features))
    # print(X_features)
    categorical_features = ['credit_history', 'savings', 'employment']

    # privileged classes
    all_privileged_classes = {"sex": [1.0],
                              "age": [1.0]}

    # protected attribute maps
    all_protected_attribute_maps = {"sex": {1.0: 'Male', 0.0: 'Female'},
                                    "age": {1.0: 'Old', 0.0: 'Young'}}
    result = StandardDataset(
            df=history_df,
            label_name=Y_features[0],
            favorable_classes=[1],
            protected_attribute_names=D_features,
            privileged_classes=[all_privileged_classes[x] for x in D_features],
            instance_weights_name=None,
            features_to_keep=['credit_history=Other','savings=<500']+Y_features+D_features,
            metadata={ 'label_maps': [{1.0: 'Good Credit', 2.0: 'Bad Credit'}],
                    'protected_attribute_maps': [all_protected_attribute_maps[x]
                                    for x in D_features]})

    aligned_dataset = dataset_history.align_datasets(result)

    return aligned_dataset

In [17]:
# TEST THAT THE APPENDING WORKS

history_data = dataset_orig_epochs[0]
for epoch_ind in range(1,len(dataset_orig_epochs)):
    # print("Size of epoch: ", dataset_orig_epochs[epoch_ind].features.shape)
    history_data = append_dataset_history(history_data,dataset_orig_epochs[epoch_ind], ["age"])
    # print("Size of history: ", history_data.features.shape)

Size of epoch:  (500, 3)
Size of history:  (1000, 3)


In [18]:
# credit_history=Other is the meaningful features
MEANINGFUL_FEATURE_IND = 1

ADV_MOD_PROB_MEAN = 0.8
ADV_MOD_PROB_STD = 0.05
DIS_MOD_PROB_MEAN = 0.5
DIS_MOD_PROB_STD = 0.2
MOD_THRESHOLD = 0.7

def strategizing_agents(data_epoch, model, FAVORABLE_LABEL_MOD_THRESHOLD,PER_EPOCH_PENALTY,MODIFY_GAIN):
    # advantaged agents have higher distribution of flipping coin to modify
    # if successful, can choose which feature to modify

    # get scaled data from agent
    scale_orig = StandardScaler()
    X_train = scale_orig.fit_transform(data_epoch.features)
    y_train = data_epoch.labels.ravel()

    # FAVORABLE_LABEL_MOD_THRESHOLD = 1
    # PER_EPOCH_PENALTY = 1
    # MODIFY_GAIN = 1

    unfavorable_classification_count = 0
    mod_count = 0
    new_classication_count = 0
    adv_agent_new_classificaton_count = 0
    adv_agent_count = 0
    meaningful_mod_count = 0
    favorable_label_change_count = 0
    unfavorable_label_change_count = 0


    for agent_ind in range(data_epoch.features.shape[0]):
        # print(f"Agent {agent_ind}")
        # print(X_train[agent_ind])
        if X_train[agent_ind][0] > 0:
            adv_agent_count +=1

        # penalize everyone by the meaningful feature (credit history)
        # subtract by a standard deviation
        for feat_ind in range(1,data_epoch.features.shape[1]):
            X_train[agent_ind][feat_ind] -= PER_EPOCH_PENALTY
        
        # original classification
        agent_classification = model.predict([X_train[agent_ind]])
        # print("classify: ",agent_classification)
        
        # if bad credit, agent will try to modify feature
        # advantaged agent more likely to be able to modify
        if agent_classification == 2: # bad credit
            unfavorable_classification_count+=1
            # print("bad credit")
            # under 25, disadvantaged
            if data_epoch.unprivileged_protected_attributes[0][0] == 0: 
                prob_change_feature = np.random.normal(DIS_MOD_PROB_MEAN,DIS_MOD_PROB_STD)
            else: 
                prob_change_feature = np.random.normal(ADV_MOD_PROB_MEAN,ADV_MOD_PROB_STD)
        else:
            prob_change_feature = 0
        
        # if allowed to modify
        if prob_change_feature > MOD_THRESHOLD:
            mod_count += 1
            # randomly iterate through features to modify
            # if successfully re-classifies, then exit out
            feature_indices = list(range(1,len(data_epoch.feature_names)))
            random.shuffle(feature_indices)

            for feature_ind in feature_indices:
                # print(data_history.feature_names[feature_ind])
                # if data_history.feature_names[feature_ind] is not data_history.protected_attribute_names[0]:
                #     if X_train[agent_ind][feature_ind] < 0:
                
                # improve one of the features
                try_new_train = list(X_train[agent_ind])
                try_new_train[feature_ind] = min(1.5,try_new_train[feature_ind]+MODIFY_GAIN)
                new_classify = model.predict([try_new_train])

                # check if feature modification did in help improve classification
                if new_classify < agent_classification:
                    # store as the X_train
                    data_epoch.features[agent_ind] = try_new_train 

                    # counters
                    new_classication_count+=1
                    if X_train[agent_ind][0] > 0:
                        adv_agent_new_classificaton_count+=1
                    if feature_ind == MEANINGFUL_FEATURE_IND:
                        meaningful_mod_count += 1
                    
                    # print(new_classify)
                    # print(try_new_train)

                    # if classification has been improved, leave for-loop
                    break
    
        # change the true labels based on feature mod; important for future training
        original_label = int(data_epoch.labels[agent_ind])
        # print("original label: ", original_label)
        # print("new meaningful feature val: ", data_epoch.features[agent_ind][MEANINGFUL_FEATURE_IND])
        if data_epoch.features[agent_ind][MEANINGFUL_FEATURE_IND] > FAVORABLE_LABEL_MOD_THRESHOLD:
            data_epoch.labels[agent_ind] = 1
            if data_epoch.labels[agent_ind] != original_label:
                favorable_label_change_count += 1
        else: 
            data_epoch.labels[agent_ind] = 2
            if data_epoch.labels[agent_ind] != original_label:
                unfavorable_label_change_count += 1

    # print("adv_agent_count", adv_agent_count)
    # print("unfavorable_classification_count: ",unfavorable_classification_count)
    # print("mod_count: ", mod_count)
    # print("new_classication_count: ", new_classication_count)
    # print("adv_agent_new_classificaton_count: ",adv_agent_new_classificaton_count)
    # print("meaningful_mod_count: ",meaningful_mod_count)
    # print("favorable_label_change_count: ",favorable_label_change_count)
    # print("unfavorable_label_change_count: ",unfavorable_label_change_count)

    return data_epoch

    
# SEPARATELY NEED TO RETRAIN MODEL AND APPEND HISTORY
    
                    


In [37]:
# each step uses a classifier
# strategizes next epoch 
# show fairness
def iterative_gaming_sim(fair_algo, gaming):
    dataset_orig = load_dropped_data_german(['age'])

    privileged_groups = [{'age': 1}]
    unprivileged_groups = [{'age': 0}]

    NUM_TRIALS = 10
    NUM_EPOCHS = 4

    avg_metrics = [{
        "Balanced accuracy":0,
        "Statistical parity difference":0,
        "Disparate impact":0,
        "Equal opportunity difference":0,
        "Consistency (individual fairness)":0
    }]*(NUM_EPOCHS-1)
    
    for trial in range(NUM_TRIALS):
        # split data into epochs, each with a different group of agents
        
        epochs = dataset_orig.split(NUM_EPOCHS, shuffle=True)
        # print("Size of epoch: ", dataset_orig_epochs[0].features.shape)
        
        nfair_ngame_history = epochs[0]

        for epoch in range(NUM_EPOCHS-1):
            # display(Markdown(f"### Epoch {epoch}/{NUM_EPOCHS}"))
            if fair_algo == "None":
                model,metrics = no_fairness_train_and_classify(nfair_ngame_history,epochs[epoch+1])
            elif fair_algo == "RW":
                model,metrics = RW_train_and_classify(nfair_ngame_history,epochs[epoch+1])
            elif fair_algo == "ROC":
                model,metrics = ROC_train_and_classify(nfair_ngame_history,epochs[epoch+1])
            elif fair_algo == "RW ROC":
                model,metrics = RW_ROC_train_and_classify(nfair_ngame_history,epochs[epoch+1])
            else:
                print("invalid fairness algo")
            
            if gaming:
                next_epoch = strategizing_agents(epochs[epoch+1],model,1,1,1)
            else:
                next_epoch = epochs[epoch+1]
            nfair_ngame_history = append_dataset_history(nfair_ngame_history, next_epoch, ['age'])

            avg_metrics[epoch]["Balanced accuracy"] += metrics["Balanced accuracy"]
            avg_metrics[epoch]["Statistical parity difference"] += metrics["Statistical parity difference"]
            avg_metrics[epoch]["Disparate impact"] += metrics["Disparate impact"]
            avg_metrics[epoch]["Equal opportunity difference"] += metrics["Equal opportunity difference"]
            avg_metrics[epoch]["Consistency (individual fairness)"] += metrics["Consistency (individual fairness)"]

            # print(avg_metrics[epoch-1])

    for epoch in range(NUM_EPOCHS-1):
        avg_metrics[epoch]["Balanced accuracy"] /= NUM_TRIALS
        avg_metrics[epoch]["Statistical parity difference"] /= NUM_TRIALS
        avg_metrics[epoch]["Disparate impact"] /= NUM_TRIALS
        avg_metrics[epoch]["Equal opportunity difference"] /= NUM_TRIALS
        avg_metrics[epoch]["Consistency (individual fairness)"] /= NUM_TRIALS

    for epoch in range(len(avg_metrics)):
        display(Markdown(f"### Epoch {epoch} fairness metrics"))
        for k in avg_metrics[epoch]:
            print("%s = %.4f" % (k, avg_metrics[epoch][k]))
    
    return avg_metrics




In [38]:
import csv 

ng_nf_results= iterative_gaming_sim("None", True)
# with open('ng_nf_results.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
#     w = csv.DictWriter(f, ng_nf_results.keys())
#     w.writeheader()
#     w.writerow(ng_nf_results)

rw_results = iterative_gaming_sim("RW", True)
# with open('rw_results.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
#     w = csv.DictWriter(f, rw_results.keys())
#     w.writeheader()
#     w.writerow(rw_results)

roc_results = iterative_gaming_sim("ROC", True)
# with open('roc_results.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
#     w = csv.DictWriter(f, roc_results.keys())
#     w.writeheader()
#     w.writerow(roc_results)

rw_roc_results = iterative_gaming_sim("RW ROC", True)
# with open('rw_roc_results.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
#     w = csv.DictWriter(f, rw_roc_results.keys())
#     w.writeheader()
#     w.writerow(rw_roc_results)


['credit_history', 'savings', 'employment']
0      A34
1      A32
2      A34
3      A32
4      A33
      ... 
995    A32
996    A32
997    A32
998    A32
999    A34
Name: credit_history, Length: 1000, dtype: object
0          Other
1      None/Paid
2          Other
3      None/Paid
4          Delay
         ...    
995    None/Paid
996    None/Paid
997    None/Paid
998    None/Paid
999        Other
Name: credit_history, Length: 1000, dtype: object


/Users/jessicachen/opt/anaconda3/envs/226/lib/python3.8/site-packages/aif360/metrics/dataset_metric.py:82: RuntimeWarning: invalid value encountered in scalar divide
  return metric_fun(privileged=False) / metric_fun(privileged=True)
/Users/jessicachen/opt/anaconda3/envs/226/lib/python3.8/site-packages/aif360/metrics/dataset_metric.py:82: RuntimeWarning: invalid value encountered in scalar divide
  return metric_fun(privileged=False) / metric_fun(privileged=True)
/Users/jessicachen/opt/anaconda3/envs/226/lib/python3.8/site-packages/aif360/metrics/dataset_metric.py:82: RuntimeWarning: invalid value encountered in scalar divide
  return metric_fun(privileged=False) / metric_fun(privileged=True)
/Users/jessicachen/opt/anaconda3/envs/226/lib/python3.8/site-packages/aif360/metrics/dataset_metric.py:82: RuntimeWarning: invalid value encountered in scalar divide
  return metric_fun(privileged=False) / metric_fun(privileged=True)
/Users/jessicachen/opt/anaconda3/envs/226/lib/python3.8/site-pac

### Epoch 0 fairness metrics

Balanced accuracy = 0.0155
Statistical parity difference = -0.0044
Disparate impact = nan
Equal opportunity difference = -0.0041
Consistency (individual fairness) = 0.0190


### Epoch 1 fairness metrics

...

Balanced accuracy = 0.0300
Statistical parity difference = -0.0023
Disparate impact = 0.0270
Equal opportunity difference = 0.0000
Consistency (individual fairness) = 0.0184


### Epoch 1 fairness metrics

Balanced accuracy = 0.0300
Statistical parity difference = -0.0023
Disparate impact = 0.0270
Equal opportunity difference = 0.0000
Consistency (individual fairness) = 0.0184


### Epoch 2 fairness metrics

Balanced accuracy = 0.0300
Statistical parity difference = -0.0023
Disparate impact = 0.0270
Equal opportunity difference = 0.0000
Consistency (individual fairness) = 0.0184
